# Using ML - ML fundamentals

The `bigframes.ml` module implements Scikit-Learn's machine learning API in
BigQuery DataFrames. It exposes BigQuery's ML capabilities in a simple, popular
API that works seamlessly with the rest of the BigQuery DataFrames API.

In [ ]:
# Lets load some test data to use in this tutorial
import bigframes.pandas

df = bigframes.pandas.read_gbq("bigquery-public-data.ml_datasets.penguins")
df = df.dropna()

# Temporary workaround: lets name our index so it isn't lost BigQuery DataFrame
# currently drops unnamed indexes when round-tripping through pandas, which
# some ML APIs do to route around missing functionality
df.index.name = "penguin_id"

df

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
penguin_id,,,,,,,
0,Adelie Penguin (Pygoscelis adeliae),Dream,36.6,18.4,184.0,3475.0,FEMALE
1,Adelie Penguin (Pygoscelis adeliae),Dream,39.8,19.1,184.0,4650.0,MALE
2,Adelie Penguin (Pygoscelis adeliae),Dream,40.9,18.9,184.0,3900.0,MALE
3,Chinstrap penguin (Pygoscelis antarctica),Dream,46.5,17.9,192.0,3500.0,FEMALE
4,Adelie Penguin (Pygoscelis adeliae),Dream,37.3,16.8,192.0,3000.0,FEMALE
5,Adelie Penguin (Pygoscelis adeliae),Dream,43.2,18.5,192.0,4100.0,MALE
6,Chinstrap penguin (Pygoscelis antarctica),Dream,46.9,16.6,192.0,2700.0,FEMALE
7,Chinstrap penguin (Pygoscelis antarctica),Dream,50.5,18.4,200.0,3400.0,FEMALE
8,Chinstrap penguin (Pygoscelis antarctica),Dream,49.5,19.0,200.0,3800.0,MALE


## Data split

Part of preparing data for a machine learning task is splitting it into subsets for training and testing, to ensure that the solution is not overfitting. Most commonly this is done with `bigframes.ml.model_selection.train_test_split` like so:

In [2]:
# In this example, we're doing supervised learning, where we will learn to predict
# output variable `y` from input features `X`
X = df[['island', 'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'sex', 'species']]
y = df[['body_mass_g']] 

from bigframes.ml.model_selection import train_test_split

# This will split X and y into test and training sets, with 20% of the rows in the test set,
# and the rest in the training set
train_X, test_X, train_y, test_y = train_test_split(
  X, y, test_size=0.2)

# Show the shape of the data after the split
print(f"""train_X shape: {train_X.shape}
test_X shape: {test_X.shape}
train_y shape: {train_y.shape}
test_y shape: {test_y.shape}""")

train_X shape: (267, 6)
test_X shape: (67, 6)
train_y shape: (267, 1)
test_y shape: (67, 1)


In [3]:
# If we look at the data, we can see that random rows were selected for
# each side of the split
test_X.head(5)

,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,sex,species
penguin_id,,,,,,
125,Biscoe,39.7,18.9,184.0,MALE,Adelie Penguin (Pygoscelis adeliae)
260,Biscoe,50.4,15.7,222.0,MALE,Gentoo penguin (Pygoscelis papua)
93,Dream,41.1,18.1,205.0,MALE,Adelie Penguin (Pygoscelis adeliae)
255,Biscoe,44.5,14.7,214.0,FEMALE,Gentoo penguin (Pygoscelis papua)
71,Dream,32.1,15.5,188.0,FEMALE,Adelie Penguin (Pygoscelis adeliae)


In [4]:
# Note that this matches the rows in test_X
test_y.head(5)

,body_mass_g
penguin_id,
125,3550.0
260,5750.0
93,4300.0
255,4850.0
71,3050.0


## Estimators

Following Scikit-Learn, all learning components are "estimators"; objects that can learn from training data and then apply themselves to new data. Estimators share the following patterns:

- a constructor that takes a list of parameters
- a standard string representation that shows the class name and all non-default parameters, e.g. `LinearRegression(fit_intercept=False)`
- a `.fit(..)` method to fit the estimator to training data

There estimators can be further broken down into two main subtypes:

### Transformers

Transformers are estimators that are used to prepare data for consumption by other estimators ('preprocessing'). In addition to `.fit(...)`, the transformer implements a `.transform(...)` method, which will apply a transformation based on what was computed during `.fit(..)`. With this pattern dynamic preprocessing steps can be applied to both training and test/production data consistently.

An example of a transformer is `bigframes.ml.preprocessing.StandardScaler`, which rescales a dataset to have a mean of zero and a standard deviation of one:

In [5]:
from bigframes.ml.preprocessing import StandardScaler

# StandardScaler will only work on numeric columns
numeric_columns = ["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm"]

scaler = StandardScaler()
scaler.fit(train_X[numeric_columns])

# Now, standardscaler should transform the numbers to have mean of zero
# and standard deviation of one:
scaler.transform(train_X[numeric_columns])

,scaled_culmen_length_mm,scaled_culmen_depth_mm,scaled_flipper_length_mm
penguin_id,,,
23,-1.310231,0.687253,-0.568262
83,2.575955,0.329548,-1.432429
195,0.816172,-1.407876,0.151877
264,1.109469,-0.436962,2.096254
237,0.816172,-1.305674,0.872017
261,0.614531,-0.947969,1.520142
253,0.321234,-1.663379,0.944031
164,0.101261,-0.743566,1.160073
293,-0.686975,-0.079257,-1.792499


In [6]:
# We can then repeat this transformation on new data
scaler.transform(test_X[numeric_columns])

,scaled_culmen_length_mm,scaled_culmen_depth_mm,scaled_flipper_length_mm
penguin_id,,,
125,-0.77863,0.891656,-1.216387
260,1.182794,-0.743566,1.520142
93,-0.521995,0.482851,0.295905
255,0.101261,-1.254574,0.944031
71,-2.171791,-0.845768,-0.928332
327,-0.521995,0.227347,-1.360415
338,-0.980272,2.066973,-0.71229
68,1.29278,0.840556,0.151877
17,1.219456,1.14716,-0.568262


#### Composing transformers

To process data where different columns need different preprocessors, `bigframes.composition.ColumnTransformer` can be employed:

In [7]:
from bigframes.ml.compose import ColumnTransformer
from bigframes.ml.preprocessing import OneHotEncoder

# Create an aggregate transform that applies StandardScaler to the numeric columns,
# and OneHotEncoder to the string columns
preproc = ColumnTransformer([
    ("scale", StandardScaler(), ["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm"]),
    ("encode", OneHotEncoder(), ["species", "sex", "island"])])

# Now we can fit all columns of the training data
preproc.fit(train_X)

processed_train_X = preproc.transform(train_X)
processed_test_X = preproc.transform(test_X)

processed_train_X

,onehotencoded_island,scaled_culmen_length_mm,scaled_culmen_depth_mm,scaled_flipper_length_mm,onehotencoded_sex,onehotencoded_species
penguin_id,,,,,,
23,"[{'index': 2, 'value': 1.0}]",-1.310231,0.687253,-0.568262,"[{'index': 1, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
83,"[{'index': 2, 'value': 1.0}]",2.575955,0.329548,-1.432429,"[{'index': 1, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
195,"[{'index': 1, 'value': 1.0}]",0.816172,-1.407876,0.151877,"[{'index': 1, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
264,"[{'index': 1, 'value': 1.0}]",1.109469,-0.436962,2.096254,"[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
237,"[{'index': 1, 'value': 1.0}]",0.816172,-1.305674,0.872017,"[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
261,"[{'index': 1, 'value': 1.0}]",0.614531,-0.947969,1.520142,"[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
253,"[{'index': 1, 'value': 1.0}]",0.321234,-1.663379,0.944031,"[{'index': 1, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
164,"[{'index': 1, 'value': 1.0}]",0.101261,-0.743566,1.160073,"[{'index': 0, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
293,"[{'index': 3, 'value': 1.0}]",-0.686975,-0.079257,-1.792499,"[{'index': 1, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"


### Predictors

Predictors are estimators that learn and make predictions. In addition to `.fit(...)`, the predictor implements a `.predict(...)` method, which will use what was learned during `.fit(...)` to predict some output.

Predictors can be further broken down into two categories:

#### Supervised predictors

Supervised learning is when we train a model on input-output pairs, and then ask it to predict the output for new inputs. An example of such a predictor is `bigframes.ml.linear_models.LinearRegression`.

In [8]:
from bigframes.ml.linear_model import LinearRegression

linreg = LinearRegression()

# Learn from the training data how to predict output y
linreg.fit(processed_train_X, train_y)

# Predict y for the test data
predicted_test_y = linreg.predict(processed_test_X)

predicted_test_y

,predicted_body_mass_g
penguin_id,
255,4789.541771
158,4642.382347
230,3562.484258
218,5321.625743
297,3926.179647
93,4158.344927
338,4093.191646
267,5590.390771
275,3623.482531


#### Unsupervised predictors

In unsupervised learning, there are no known outputs in the training data, instead the model learns on input data alone and predicts something else. An example of an unsupervised predictor is `bigframes.ml.cluster.KMeans`, which learns how to fit input data to a target number of clusters.

In [9]:
from bigframes.ml.cluster import KMeans

kmeans = KMeans(n_clusters=4)

kmeans.fit(processed_train_X)

kmeans.predict(processed_test_X)

,CENTROID_ID
penguin_id,
17,1
230,2
291,4
177,3
206,2
331,2
18,2
218,4
116,2


## Pipelines

Transfomers and predictors can be chained into a single estimator component using `bigframes.ml.pipeline.Pipeline`:

In [10]:
from bigframes.ml.pipeline import Pipeline

pipeline = Pipeline([
  ('preproc', preproc),
  ('linreg', linreg)
])

# Print our pipeline
pipeline

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  ['culmen_length_mm',
                                                   'culmen_depth_mm',
                                                   'flipper_length_mm']),
                                                 ('encode', OneHotEncoder(),
                                                  ['species', 'sex',
                                                   'island'])])),
                ('linreg', LinearRegression())])

The pipeline simplifies the workflow by applying each of its component steps automatically:

In [11]:
pipeline.fit(train_X, train_y)

predicted_test_y = pipeline.predict(test_X)
predicted_test_y

,predicted_body_mass_g
penguin_id,
125,3911.489556
260,5477.221344
93,4158.347237
255,4789.543554
71,3153.676672
327,3358.229939
338,4093.194446
68,4133.390538
17,4013.257221


In the backend, a pipeline will actually be compiled into a single model with an embedded TRANSFORM step.

## Evaluating results

Some models include a convenient `.score(X, y)` method for evaulation with a preset accuracy metric:

In [12]:
# In the case of a pipeline, this will be equivalent to calling .score on the contained LinearRegression
pipeline.score(test_X, test_y)

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,223.862457,73619.165369,0.004765,209.607289,0.888368,0.901838


For a more general approach, the library `bigframes.ml.metrics` is provided:

In [13]:
from bigframes.ml.metrics import r2_score

r2_score(test_y, predicted_test_y)

0.8883676417577451

## Save/Load to BigQuery

Estimators can be saved to BigQuery as BQML models, and loaded again in future

In [14]:
# Replace with a path where you have permission to save a model
model_name = "bigframes-dev.bqml_tutorial.penguins_model"

linreg.to_gbq(model_name, replace=True)

LinearRegression()

In [15]:
# WARNING - until b/281709360 is fixed & pipeline is updated, pipelines will load as models,
# and details of their transform steps will be lost (the loaded model will behave the same)
bigframes.pandas.read_gbq_model(model_name)

LinearRegression()